In [1]:
import pandas as pd
import re
import random
mov = open('movies.dat', "r")



titles =[]
years = []
genres = []
genres2 = []
Id = []

title_pattern = '\w+.{1,8}\w+.{1,8}\w+.{1,8}\\('
Id_pattern = '\d+'
year_pattern = '\\(\d+\\)'
genre_pattern = '\\:\w+\\|?\w+'

for i, line in enumerate(mov):
    id = re.search(Id_pattern, line).group()
    Id.append(int(id))
    tit = re.search(title_pattern, line).group()
    title = ''.join(filter(lambda x: not x.isdigit(), tit))
    titles.append(title[2: len(title) - 2])
    year = re.search(year_pattern, line).group()
    years.append(int(year[1 : (len(year) - 1)]))
    genre = re.search(genre_pattern, line[6:]).group()
    genres.append(genre[1:])
    genres2.append([id] + genre[1:].split('|'))
        
df = pd.DataFrame(index = Id)
df['MovId'] = Id
df["Title"] = titles
df['Year'] = years
df['genre'] = genres

genre_dict = {}
for genre in genres2:
    for gen in genre[1:]:
        if gen not in list(genre_dict.keys()):
            genre_dict[gen] = [genre[0]]
        else:
            genre_dict[gen].append(genre[0])
for genre in list(genre_dict.keys()):
    df[genre] = [0] * len(df)
    for i in genre_dict[genre]:
        df.loc[int(i) ,genre] = 1

df.reset_index(inplace = True)

In [2]:
ratings_f = open('ratings.dat', "r")
mid = []
rate = []
uid = []
ratings = pd.DataFrame()

for i, line in enumerate(ratings_f):
    id = re.search('\d+\\:',line).group()
    mov_id = re.search('\\:\d+\\:',line).group()
    rating = re.search('\\:\d\\:',line).group()
    uid.append(int(id[:(len(id)-1)]))
    mid.append(int(mov_id[1 :len(mov_id)-1]))
    rate.append(int(rating[1]))
        
ratings['Uid'] = uid
ratings['mid']= mid
ratings['rate'] = rate
final_r = ratings.pivot(index = 'mid', columns = 'Uid', values ='rate' )
final_r.fillna(0, inplace = True)

In [25]:
df

,index,MovId,Title,Year,genre,Animation,Children,Adventure,Comedy,Romance,...,Horror,Thriller,Sci,Documentary,War,Mystery,Musical,Film,Western,Fantasy
0,1,1,Toy Story,1995,Animation|Children,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,2,Jumanji,1995,Adventure|Children,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,3,Grumpier Old Men,1995,Comedy|Romance,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
3,4,4,Waiting to Exhale,1995,Comedy|Drama,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,5,5,ther of the Bride Part II,1995,Comedy,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3878,3948,3948,Meet the Parents,2000,Comedy,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3879,3949,3949,Requiem for a Dream,2000,Drama,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3880,3950,3950,Tigerland,2000,Drama,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3881,3951,3951,Two Family House,2000,Drama,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
final_r = final_r.loc[:, ratings.groupby('Uid')['rate'].agg('count') >50]

In [4]:
final_r = final_r[ratings.groupby('mid')['rate'].agg('count') > 10]

In [4]:
final_r.index

Int64Index([   1,    2,    3,    4,    5,    6,    7,    8,    9,   10,
            ...
            3942, 3943, 3945, 3946, 3947, 3948, 3949, 3950, 3951, 3952],
           dtype='int64', name='mid', length=3233)

In [13]:
final_r

Uid,1,2,3,5,6,8,9,10,11,13,...,6026,6030,6031,6032,6033,6035,6036,6037,6039,6040
mid,,,,,,,,,,,,,,,,,,,,,
1,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3948,0.0,0.0,0.0,0.0,0.0,0.0,3.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3949,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3950,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
from scipy.sparse import csr_matrix
csr_data = csr_matrix(final_r.values)
final_r.reset_index(inplace=True)

In [6]:
from sklearn.neighbors import NearestNeighbors
knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-1)
knn.fit(csr_data)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=20)

In [14]:
distances, neighbors = knn.kneighbors(csr_data[234], n_neighbors=10)
print(distances)
print(neighbors)
print(sorted(distances[0]))
print(sorted(neighbors[0]))

[[5.55111512e-15 5.14402105e-01 5.25912326e-01 5.36417128e-01
  5.50390019e-01 5.64681334e-01 5.71424357e-01 5.81234470e-01
  5.86075106e-01 5.88698325e-01]]
[[ 234  876   52   16  460  320 1925  542 1829 2772]]
[5.551115123125783e-15, 0.5144021050157932, 0.5259123261069205, 0.5364171282803237, 0.5503900188067192, 0.5646813343624659, 0.5714243572817572, 0.5812344695093861, 0.5860751057015665, 0.5886983245194897]
[16, 52, 234, 320, 460, 542, 876, 1829, 1925, 2772]


In [16]:
def get_rec(movie):
    mov_ind = df[df["Title"].str.contains(movie)]
    if not mov_ind.empty:
        mov_ind = mov_ind.index[0]
        movie_id = df.iloc[mov_ind]['MovId']
        mov_id = final_r[final_r['mid'] == movie_id].index[0]
        distances, neighbors = knn.kneighbors(csr_data[mov_id], n_neighbors=10)
        print(neighbors)
        rec_mov_ind =  sorted(neighbors[0])
        print(rec_mov_ind)
        movies = []
        Id = []
        genres = []
        for index in rec_mov_ind:
            mov = df.iloc[index]['Title']
            mid = df.iloc[index]['MovId']
            genre = df.iloc[index]['genre']
            movies.append(mov)
            Id.append(mid)
            genres.append(genre)
        rec = pd.DataFrame()
        rec['title'] = movies[]
        rec['Id'] = Id[]
        rec['genres'] = genres[]

        return rec

In [26]:
get_rec('Toy Story')

[[   0 1010 2547  527 1015 1885  943 1253  230  319]]
[0, 230, 319, 527, 943, 1010, 1015, 1253, 1885, 2547]


,title,Id,genres
0,Exotica,233,Drama
1,Swimming with Sharks,322,Comedy|Drama
2,"Secret Garden, The",531,Children
3,Bringing Up Baby,955,Comedy
4,e Pooh and the Blustery Day,1023,Animation|Children
5,Mary Poppins,1028,Children
6,Down by Law,1273,Comedy|Drama
7,Rocky,1954,Action|Drama
8,Dick Tracy,2616,Action|Crime


In [18]:
knn2 = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=100, n_jobs=-1)
knn2.fit(df.drop(['Title','Year','genre'], axis=1))

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=100)

In [49]:
df.drop(['Title','Year','genre'], axis=1).iloc[0].values

array([1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=int64)

In [24]:
def get_rec_genre(movie):
    mov_ind = df[df["Title"].str.contains(movie)]
    if not mov_ind.empty:
        mov_ind = mov_ind.index[0]
        distances, neighbors = knn2.kneighbors(df.drop(['Title','Year','genre'], axis=1), n_neighbors=50)
        random.shuffle(neighbors[mov_ind])
        return df.iloc[neighbors[mov_ind][:20]]

In [25]:
get_rec_genre('Toy Story')

,index,MovId,Title,Year,genre,Animation,Children,Adventure,Comedy,Romance,...,Horror,Thriller,Sci,Documentary,War,Mystery,Musical,Film,Western,Fantasy
236,239,239,"Goofy Movie, A",1995,Animation|Children,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1011,1024,1024,"Three Caballeros, The",1945,Animation|Children,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
47,48,48,Pocahontas,1995,Animation|Children,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,2,Jumanji,1995,Adventure|Children,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
360,364,364,"Lion King, The",1994,Animation|Children,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
156,158,158,Casper,1995,Adventure|Children,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1010,1023,1023,e Pooh and the Blustery Day,1968,Animation|Children,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
235,238,238,me: The Adventures of Yellow Dog,1995,Adventure|Children,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
310,313,313,"Swan Princess, The",1994,Animation|Children,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
876,888,888,I: The Time of the Great Giving,1995,Animation|Children,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
